01/21/2022
### in v11, DCAP_M is used as load
12/25/2021
### in v10, CCO_unit_{i}S has been corrected. In v9, CCO_unit_{i}S is not correct, there is an internal INV in v9
12/19/2021
### in v9, CCO now has DCAP as load
12/11/2021
### in v8, CCO size now can change

12/02/2021
### in v7, output Vctrl and CCO_ring X[0] so that can set initial condition during post-simulation

11/28/2021
## in v6 CCO_ring has 2 levels now, CCO_ring, CCO_unit, CCO.  e.g. 120N = 15N*8
## gen the top level of CCO DSM including resistor
## Compared with v4, v5 resistor has [25, 10, 5, 1]

In [1]:
import sys
import re
import os
#import linecache
#import numpy as np 
#import codecs
#import numpy as np
#import matplotlib.pyplot as plt
from icecream import ic
import time

In [47]:
Time = time.strftime("%m-%d-%Y", time.localtime())

In [48]:
# make new dir
def mkdir(path):
 
    folder = os.path.exists(path)
 
    if not folder:                   
        os.makedirs(path)            
        print ("---  new folder...  ---")
        print ("---  OK  ---")
 
    else:
        print ("---  There is this folder!  ---")




In [49]:
CCO_size_lib = ["CCO82", "CCO124", "CCO164", "CCO206", "CCO248"]
CCO_load_lib = ["DCAP0", "DCAP1", "DCAP4", "DCAP8", "DCAP16", "DCAP32", "DCAP64"]
#ic(CCO_size[1])

In [50]:
Time

'02-14-2022'

In [51]:
N = 31
S = 3
CCO_size_num = 0
CCO_load_num = 2
CCO_size = CCO_size_lib[CCO_size_num]
CCO_load = CCO_load_lib[CCO_load_num]
RIN = 3
Rdac = 3
version = 1

New_file = str("CCO_DSM_{i}N_{j}S_{k}kRin_v{t}_{m}".format(i = N, j = S, k = RIN, t = version, m = Time))
file = "./{i}".format(i = New_file)
mkdir(file)             

---  new folder...  ---
---  OK  ---


In [52]:
ic(CCO_size)
ic(CCO_load)

ic| CCO_size: 'CCO82'
ic| CCO_load: 'DCAP4'


'DCAP4'

In [53]:

CCO_ring = str("./{fn}/CCO_ring_{i}N.txt".format(fn = New_file, i=N))
with open("./{fn}/CCO_ring_{i}N.txt".format(fn = New_file, i=N), "w") as f:
    f.writelines('//CCO_ring_{i}N \n'.format(i = N))
    f.writelines('//')
    f.writelines(Time)
    f.writelines('\n')
    f.writelines('module CCO_ring_{i}N(X, XN, VCTRL, VSS); \n'.format(i = N))
    f.writelines('inout VCTRL, VSS; \n')
    f.writelines('output [{i}:0] X; \n'.format(i = N-1))
    f.writelines('output [{i}:0] XN; \n'.format(i = N-1))
    f.writelines('\n')
    f.writelines('CCO_unit_{j}S I{i} (.OUT(XN[{i}]), .OUTN(X[{i}]), .VCTRL(VCTRL), .VSS(VSS), .IN(X[{k}]), .INN(XN[{k}])); \n'.format(j=S, i = 0, k = N-1))
    for i in range(1, N):
            f.writelines('CCO_unit_{j}S I{i} (.OUT(XN[{i}]), .OUTN(X[{i}]), .VCTRL(VCTRL), .VSS(VSS), .IN(X[{k}]), .INN(XN[{k}]));'.format(j=S, i = i, k = i-1))
            f.writelines('\n')
    f.writelines('endmodule \n')
   

In [54]:
CCO_unit = str("./{fn}/CCO_unit_{i}S.txt".format(fn = New_file, i=S))
with open("./{fn}/CCO_unit_{i}S.txt".format(fn = New_file, i=S), "w") as f:
    f.writelines('//CCO_unit_{i}S \n'.format(i = S))
    f.writelines('//')
    f.writelines(Time)
    f.writelines('\n')
    f.writelines('module CCO_unit_{i}S(IN, INN, VCTRL, VSS, OUT, OUTN); \n'.format(i = S))
    f.writelines('inout VCTRL, VSS; \n')
    f.writelines('input IN, INN; \n')
    f.writelines('output OUT, OUTN; \n')
    f.writelines('wire [{i}:0] X; \n'.format(i = S-2))
    f.writelines('wire [{i}:0] XN; \n'.format(i = S-2))
    f.writelines('\n')
    f.writelines('{f2} I{i} (.OUT(XN[{i}]), .OUTN(X[{i}]), .VCTRL(VCTRL), .VSS(VSS), .IN(IN), .INN(INN)); \n'.format(f2 = CCO_size, i = 0, k = S-1))
    for i in range(0, S-2):
            f.writelines('{f2} I{k} (.OUT(XN[{k}]), .OUTN(X[{k}]), .VCTRL(VCTRL), .VSS(VSS), .IN(X[{i}]), .INN(XN[{i}]));'.format(f2 = CCO_size, i = i, k = i+1))
            f.writelines('\n')
    f.writelines('{f2} I{i} (.OUT(OUT), .OUTN(OUTN), .VCTRL(VCTRL), .VSS(VSS), .IN(X[{j}]), .INN(XN[{j}])); \n'.format(f2 = CCO_size, i = S-1, j = S-2))
    f.writelines('endmodule \n')

In [55]:
CCO = str("./{fn}/{f2}.txt".format(fn = New_file, f2 = CCO_size))
with open("./{fn}/{f2}.txt".format(fn = New_file, f2 = CCO_size), "w") as f:
    f.writelines('//{f2} \n'.format( f2 = CCO_size))
    f.writelines('//')
    f.writelines(Time)
    f.writelines('\n')
    f.writelines('module {f2}(IN, INN, VCTRL, VSS, OUT, OUTN); \n'.format(f2 = CCO_size))
    f.writelines('inout VCTRL, VSS; \n')
    f.writelines('input IN, INN; \n')
    f.writelines('output OUT, OUTN; \n')
    if CCO_size_num == 0:
        INV1_type = 8
        INV2_type = 2
    elif CCO_size_num == 1:
        INV1_type = 12
        INV2_type = 4
    elif CCO_size_num ==2:
        INV1_type = 16
        INV2_type = 4
    elif CCO_size_num == 3:
        INV1_type = 20
        INV2_type = 6
    elif CCO_size_num == 4:
        INV1_type = 24
        INV2_type = 8
    else:
        print("wrong CCO_size_num")
    
    f.writelines('\n')
    
    f.writelines('INVD{i} I0(.ZN(OUTN), .VDD(VCTRL), .VSS(VSS), .I(IN)); \n'.format(i = INV1_type ))
    f.writelines('INVD{i} I1(.ZN(OUT), .VDD(VCTRL), .VSS(VSS), .I(INN)); \n'.format(i = INV1_type ))
    f.writelines('INVD{i} I2(.ZN(OUTN), .VDD(VCTRL), .VSS(VSS), .I(OUT)); \n'.format(i = INV2_type ))
    f.writelines('INVD{i} I3(.ZN(OUT), .VDD(VCTRL), .VSS(VSS), .I(OUTN)); \n'.format(i = INV2_type ))
    
    if CCO_load_num == 0:
        load_type = 0
    elif CCO_load_num == 1:
        load_type = 1
    elif CCO_load_num == 2:
        load_type = 4
    elif CCO_load_num == 3:
        load_type = 8
    elif CCO_load_num == 4:
        load_type = 16
    elif CCO_load_num == 5:
        load_type = 32
    elif CCO_load_num == 6:
        load_type = 64
    else:
        print("wrong CCO_load_num")
    
    if load_type == 0:
        print("No load DCAP")
    else:
        f.writelines('DCAP{i}_M I4(.VDD(VCTRL), .I(OUT), .VSS(VSS)); \n'.format(i = load_type ))
        f.writelines('DCAP{i}_M I5(.VDD(VCTRL), .I(OUTN), .VSS(VSS)); \n'.format(i = load_type ))
    
    f.writelines('endmodule \n')

In [56]:
BUF = str("./{fn}/BUF_{i}N.txt".format(fn = New_file, i=N))
with open("./{fn}/BUF_{i}N.txt".format(fn = New_file, i=N), "w") as f:
    f.writelines('//BUF_{i}N \n'.format(i = N))
    f.writelines('//')
    f.writelines(Time)
    f.writelines('\n')
    f.writelines('module BUF_{i}N(X, XN, VDD2, VSS, OUT, OUTN); \n'.format(i = N))
    f.writelines('inout VDD2, VSS; \n')
    f.writelines('input [{i}:0] X; \n'.format(i = N-1))
    f.writelines('input [{i}:0] XN; \n'.format(i = N-1))
    f.writelines('output [{i}:0] OUT; \n'.format(i = N-1))
    f.writelines('output [{i}:0] OUTN; \n'.format(i = N-1))
    f.writelines('\n')
    for i in range(0, N):
            f.writelines('Buffer I{i} (.OUT(OUT[{i}]), .OUTN(OUTN[{i}]), .VDD(VDD2), .VSS(VSS), .IN(X[{i}]), .INN(XN[{i}]));'.format(i = i))
            f.writelines('\n')
    f.writelines('endmodule \n')

In [57]:
SAFF = str("./{fn}/SAFF_{i}N.txt".format(fn = New_file,i=N))
with open("./{fn}/SAFF_{i}N.txt".format(fn = New_file,i=N), "w") as f:
    f.writelines('//SAFF_{i}N \n'.format(i = N))
    f.writelines('//')
    f.writelines(Time)
    f.writelines('\n')
    f.writelines('module SAFF_{i}N(INP, INN, CLK, VDD, VSS, Q); \n'.format(i = N))
    f.writelines('inout VDD, VSS; \n')
    f.writelines('input CLK; \n')
    f.writelines('input [{i}:0] INP; \n'.format(i = N-1))
    f.writelines('input [{i}:0] INN; \n'.format(i = N-1))
    f.writelines('output [{i}:0] Q; \n'.format(i = N-1))
    f.writelines('\n')
    for i in range(0, N):
            f.writelines('SAFF   I{i} (.Q(Q[{i}]), .Q_B(), .VDD(VDD), .VSS(VSS), .CLK(CLK), .INP(INP[{i}]), .INN(INN[{i}]));'.format(i = i))
            f.writelines('\n')
    f.writelines('endmodule \n')

In [58]:
SAF_XOR_DFF = str("./{fn}/SAF_XOR_DFF_{i}N.txt".format(fn = New_file, i=N))
with open("./{fn}/SAF_XOR_DFF_{i}N.txt".format(fn = New_file, i=N), "w") as f:
    f.writelines('//SAF_XOR_DFF_{i}N \n'.format(i = N))
    f.writelines('//')
    f.writelines(Time)
    f.writelines('\n')
    f.writelines('module SAF_XOR_DFF_{i}N(IN1_P, IN1_N, IN2_P, IN2_N, CLK, VDD, VSS, OUT, OUT_N); \n'.format(i = N))
    f.writelines('inout VDD, VSS; \n')
    f.writelines('input  CLK; \n')
    f.writelines('input [{i}:0] IN1_P; \n'.format(i = N-1))
    f.writelines('input [{i}:0] IN1_N; \n'.format(i = N-1))
    f.writelines('input [{i}:0] IN2_P; \n'.format(i = N-1))
    f.writelines('input [{i}:0] IN2_N; \n'.format(i = N-1))
    f.writelines('output [{i}:0] OUT; \n'.format(i = N-1))
    f.writelines('output [{i}:0] OUT_N; \n'.format(i = N-1))
    f.writelines('wire [{i}:0] A; \n'.format(i = N-1))
    f.writelines('wire [{i}:0] B; \n'.format(i = N-1))
    f.writelines('wire [{i}:0] XORT; \n'.format(i = N-1))
    f.writelines('wire [{i}:0] XORT_N; \n'.format(i = N-1))
    f.writelines('\n')
    f.writelines('SAFF_{i}N I0 (.INP(IN1_P), .INN(IN1_N), .CLK(CLK), .VDD(VDD), .VSS(VSS), .Q(A)); \n'.format(i = N))
    f.writelines('SAFF_{i}N I1 (.INP(IN2_P), .INN(IN2_N), .CLK(CLK), .VDD(VDD), .VSS(VSS), .Q(B)); \n'.format(i = N))
    
    for i in range(0, N):
            f.writelines('XOR2D1 I_XOR_{i} (.A1(A[{i}]), .A2(B[{i}]), .VDD(VDD), .VSS(VSS), .Z(XORT[{i}]));'.format(i = i))
            f.writelines('\n')
    for i in range(0, N):
            f.writelines('INVD1  I_INV_{i} (.I(XORT[{i}]), .VDD(VDD), .VSS(VSS), .ZN(XORT_N[{i}]));'.format(i = i))
            f.writelines('\n')
    f.writelines('\n') 
    for i in range(0, N):
            f.writelines('DFD1   I_DFDA_{i} (.D(XORT[{i}]), .CP(CLK), .VDD(VDD), .VSS(VSS), .Q(OUT[{i}]), .QN());'.format(i = i))
            f.writelines('\n')
    for i in range(0, N):
            f.writelines('DFD1   I_DFDB_{i} (.D(XORT_N[{i}]), .CP(CLK), .VDD(VDD), .VSS(VSS), .Q(OUT_N[{i}]), .QN());'.format(i = i))
            f.writelines('\n')
    f.writelines('\n')
    f.writelines('endmodule \n')

In [64]:
DAC_BUF = str("./{fn}/DAC_BUF_{i}N.txt".format(fn = New_file, i=N))
with open("./{fn}/DAC_BUF_{i}N.txt".format(fn = New_file, i=N), "w") as f:
    f.writelines('//DAC_BUF_{i}N \n'.format(i = N))
    f.writelines('//')
    f.writelines(Time)
    f.writelines('\n')
    f.writelines('module DAC_BUF_{i}N(OUT, OUT_N, VREFP, VREFN, INP, INN); \n'.format(i = N))
    f.writelines('inout VREFP, VREFN; \n')
    f.writelines('input [{i}:0] INP; \n'.format(i = N-1))
    f.writelines('input [{i}:0] INN; \n'.format(i = N-1))
    f.writelines('output [{i}:0] OUT; \n'.format(i = N-1))
    f.writelines('output [{i}:0] OUT_N; \n'.format(i = N-1))
    f.writelines('\n')   
    for i in range(0, N):
            f.writelines('INVD2  I_INVDA_{i} (.I(INP[{i}]), .VDD(VREFP), .VSS(VREFN), .ZN(OUT[{i}]));'.format(i = i))
            f.writelines('\n')
    f.writelines('\n')       
    for i in range(0, N):
            f.writelines('INVD2  I_INVDB_{i} (.I(INN[{i}]), .VDD(VREFP), .VSS(VREFN), .ZN(OUT_N[{i}]));'.format(i = i))
            f.writelines('\n')
    f.writelines('endmodule \n')

In [65]:
#Resistor gen
#RIN, DAC

def res_gen(z):
    res_units = [25,10,5,1]
    size = len(res_units)
    s = [0 for i in range(size)]
    for i in range(0,size):
        a = z // res_units[i]
        s[i] = a
        z = z - a * res_units[i]
    return s,res_units

#RIN = 6
RES = Rdac*N

RIN_flatten = res_gen(RIN)

RES50_size = 0 #RIN_flatten[0][0]
RES25_size = RIN_flatten[0][0]
RES10_size = RIN_flatten[0][1]
RES5_size = RIN_flatten[0][2]
RES1_size = RIN_flatten[0][3]
sum_net = sum(RIN_flatten[0])-1

RIN_file = str("./{fn}/RIN_{i}k.v".format(fn = New_file, i=RIN))
with open("./{fn}/RIN_{i}k.txt".format(fn = New_file, i=RIN), "w") as f:
    f.writelines('//RIN_{i}k \n'.format(i = RIN))
    f.writelines('//')
    f.writelines(Time)
    f.writelines('\n')
    f.writelines('module RIN_{i}k(IN, OUT); \n'.format(i = RIN))
    f.writelines('inout  IN; \n')
    f.writelines('inout  OUT; \n')
    f.writelines('\n')
    for i in range(0, RES50_size):
        if (i==0):
            f.writelines('RES50 I0 (.Z(net0), .I(IN));')
        else:
            f.writelines('RES50 I{i} (.Z(net{i}), .I(net{k}));'.format(i = i, k = i-1))
        f.writelines('\n')
        
    for i in range(0, RES25_size):
        if (i==0 and RES50_size==0):
            f.writelines('RES25 I0 (.Z(net0), .I(IN));')
        else:
            f.writelines('RES25 I{i} (.Z(net{i}), .I(net{k}));'.format(i = i+RES50_size, k = RES50_size+i-1))
        f.writelines('\n')
    
    for i in range(0, RES10_size):
        if (i==0 and RES25_size==0 and RES50_size==0):
            f.writelines('RES10 I0 (.Z(net0), .I(IN));')
        else:
            f.writelines('RES10 I{i} (.Z(net{i}), .I(net{k}));'.format(i = i+RES50_size+RES25_size, k = RES25_size+RES50_size+i-1))
        f.writelines('\n')
        
    for i in range(0, RES5_size):
        if (i==0 and RES10_size==0 and RES25_size==0 and RES50_size==0):
            f.writelines('RES5 I0 (.Z(net0), .I(IN));')
        else:
            f.writelines('RES5 I{i} (.Z(net{i}), .I(net{k}));'.format(i = i+RES50_size+RES25_size+RES10_size, k = RES10_size+RES25_size+RES50_size+i-1))
        f.writelines('\n')
        
    for i in range(0, RES1_size):
        if (i==0 and RES5_size==0 and RES10_size==0 and RES25_size==0 and RES50_size==0):
            f.writelines('RES1 I0 (.Z(net0), .I(IN));')
        else:
            f.writelines('RES1 I{i} (.Z(net{i}), .I(net{k}));'.format(i = i+RES50_size+RES25_size+RES10_size+RES5_size, k = RES5_size+RES10_size+RES25_size+RES50_size+i-1))
        f.writelines('\n')
    f.writelines('endmodule \n')
    
f1 = open('./{fn}/RIN_{i}k.txt'.format(fn = New_file, i = RIN),'r+')
f2 = open('./{fn}/RIN_{i}k.v'.format(fn = New_file, i = RIN),'w+')
str1='net{i}'.format(i=sum_net)
str2 = 'OUT'
for ss in f1.readlines():
    tt = re.sub(str1,str2,ss)
    f2.write(tt)
f1.close()
f2.close()
    
    
    
res = res_gen(RES)
RES50_size = 0#res[0][0]
RES25_size = res[0][0]
RES10_size = res[0][1]
RES5_size = res[0][2]
RES1_size = res[0][3]
sum_net = sum(res[0])-1

DAC_u = str("./{fn}/DAC_u_{i}k.v".format(fn = New_file, i=RES))
with open("./{fn}/DAC_u_{i}k.txt".format(fn = New_file, i=RES), "w") as f:
    f.writelines('//DAC_u_{i}k \n'.format(i = RES))
    f.writelines('//')
    f.writelines(Time)
    f.writelines('\n')
    f.writelines('module DAC_u_{i}k(IN, OUT); \n'.format(i = RES))
    f.writelines('inout  IN; \n')
    f.writelines('inout  OUT; \n')
    f.writelines('\n')
    for i in range(0, RES50_size):
        if (i==0):
            f.writelines('RES50 I0 (.Z(net0), .I(IN));')
        else:
            f.writelines('RES50 I{i} (.Z(net{i}), .I(net{k}));'.format(i = i, k = i-1))
        f.writelines('\n')
        
    for i in range(0, RES25_size):
        if (i==0 and RES50_size==0):
            f.writelines('RES25 I0 (.Z(net0), .I(IN));')
        else:
            f.writelines('RES25 I{i} (.Z(net{i}), .I(net{k}));'.format(i = i+RES50_size, k = RES50_size+i-1))
        f.writelines('\n')
    
    for i in range(0, RES10_size):
        if (i==0 and RES25_size==0 and RES50_size==0):
            f.writelines('RES10 I0 (.Z(net0), .I(IN));')
        else:
            f.writelines('RES10 I{i} (.Z(net{i}), .I(net{k}));'.format(i = i+RES50_size+RES25_size, k = RES25_size+RES50_size+i-1))
        f.writelines('\n')
        
    for i in range(0, RES5_size):
        if (i==0 and RES10_size==0 and RES25_size==0 and RES50_size==0):
            f.writelines('RES5 I0 (.Z(net0), .I(IN));')
        else:
            f.writelines('RES5 I{i} (.Z(net{i}), .I(net{k}));'.format(i = i+RES50_size+RES25_size+RES10_size, k = RES10_size+RES25_size+RES50_size+i-1))
        f.writelines('\n')
        
    for i in range(0, RES1_size):
        if (i==0 and RES5_size==0 and RES10_size==0 and RES25_size==0 and RES50_size==0):
            f.writelines('RES1 I0 (.Z(net0), .I(IN));')
        else:
            f.writelines('RES1 I{i} (.Z(net{i}), .I(net{k}));'.format(i = i+RES50_size+RES25_size+RES10_size+RES5_size, k = RES5_size+RES10_size+RES25_size+RES50_size+i-1))
        f.writelines('\n')
    f.writelines('endmodule \n')
    
f1 = open('./{fn}/DAC_u_{i}k.txt'.format(fn = New_file, i = RES),'r+')
f2 = open('./{fn}/DAC_u_{i}k.v'.format(fn = New_file, i = RES),'w+')
str1='net{i}'.format(i=sum_net)
str2 = 'OUT'
for ss in f1.readlines():
    tt = re.sub(str1,str2,ss)
    f2.write(tt)
f1.close()
f2.close()

DAC = str("./{fn}/DAC_{i}N_{j}k.txt".format(fn = New_file, i=N, j = RES))
with open("./{fn}/DAC_{i}N_{j}k.txt".format(fn = New_file, i=N, j = RES), "w") as f:
    f.writelines('//DAC_{i}N_{j}k \n'.format(i = N, j = RES))
    f.writelines('//')
    f.writelines(Time)
    f.writelines('\n')
    f.writelines('module DAC_{i}N_{j}k(IN, OUT); \n'.format(i = N, j = RES))
    f.writelines('inout [{i}:0] IN; \n'.format(i = N-1))
    f.writelines('inout OUT; \n'.format(i = N-1))
    f.writelines('\n')
    for i in range(0, N):
            f.writelines('DAC_u_{j}k I{i} (.OUT(OUT), .IN(IN[{i}]));'.format(i = i, j = RES))
            f.writelines('\n')
    f.writelines('endmodule \n')



In [66]:
CCO_DSM_top = str("./{fn}/CCO_DSM_{i}N_{j}S_{k}kRin_v{t}.txt".format(fn = New_file, i = N, j = S, k = RIN, t = version))
#CCO_DSM_top = str("./{fn}/CCO_DSM_top_{i}N.txt".format(fn = New_file, i=N))
with open("./{fn}/CCO_DSM_{i}N_{j}S_{k}kRin_v{t}.txt".format(fn = New_file, i = N, j = S, k = RIN, t = version), "w") as f:
    f.writelines('//CCO_DSM_top_{i}N \n'.format(i = N))
    f.writelines('//')
    f.writelines(Time)
    f.writelines('\n')
    #f.writelines('module CCO_DSM_top_{i}N(INP, INN, CLK, VDD, VDD2, VSSA, VSSD, VREFP, VREFN, OUT); \n'.format(i = N))
    f.writelines('module CCO_DSM_{i}N_{j}S_{k}kRin_v{t}(INP, INN, CLK, VDD, VBUF, VBUF2, VSSA, VREFP, OUT,IN_A,IN_AN, IN_B, IN_BN, VCTRLP, VCTRLN); \n'.format(i = N, j = S, k = RIN, t = version))
    f.writelines('inout VDD, VBUF, VBUF2, VSSA, VREFP; \n')
    f.writelines('input CLK; \n')
    f.writelines('input INP; \n')
    f.writelines('input INN; \n')
    f.writelines('output [{i}:0] OUT; \n'.format(i = N-1))
    
    f.writelines('inout IN_A; \n')
    f.writelines('inout IN_B; \n')
    f.writelines('inout IN_AN; \n')
    f.writelines('inout IN_BN; \n')
    f.writelines('inout VCTRLP; \n')
    f.writelines('inout VCTRLN; \n')
    
    f.writelines('wire [{i}:0] OUT_R; \n'.format(i = N-1))
    f.writelines('wire [{i}:0] OUT_R_N; \n'.format(i = N-1))
    f.writelines('wire [{i}:0] XA; \n'.format(i = N-1))
    f.writelines('wire [{i}:0] XB; \n'.format(i = N-1))
    f.writelines('wire [{i}:0] XAN; \n'.format(i = N-1))
    f.writelines('wire [{i}:0] XBN; \n'.format(i = N-1))
    f.writelines('wire [{i}:0] OUTA; \n'.format(i = N-1))
    f.writelines('wire [{i}:0] OUTAN; \n'.format(i = N-1))
    f.writelines('wire [{i}:0] OUTB; \n'.format(i = N-1))
    f.writelines('wire [{i}:0] OUTBN; \n'.format(i = N-1))
    f.writelines('wire [{i}:0] OUT_N; \n'.format(i = N-1))
    f.writelines('wire [{i}:0] DAC_INP; \n'.format(i = N-1))
    f.writelines('wire [{i}:0] DAC_INN; \n'.format(i = N-1))
    f.writelines('\n')
    
    f.writelines('assign In_A = XA[0]; \n')
    f.writelines('assign In_AN = XAN[0]; \n')
    f.writelines('assign In_B = XB[0]; \n')
    f.writelines('assign In_BN = XBN[0]; \n')
    f.writelines('\n')
    
    f.writelines('RIN_{i}k I0  (.OUT(VCTRLP), .IN(INP)); \n'.format(i = RIN))
    f.writelines('RIN_{i}k I1  (.OUT(VCTRLN), .IN(INN)); \n'.format(i = RIN))
    
    f.writelines('CCO_ring_{i}N I2  (.X(XA), .XN(XAN), .VCTRL(VCTRLP), .VSS(VSSA)); \n'.format(i = N))
    f.writelines('CCO_ring_{i}N I3 (.X(XB), .XN(XBN), .VCTRL(VCTRLN), .VSS(VSSA)); \n'.format(i = N))
    f.writelines('BUF_{i}N I4 (.X(XA), .XN(XAN), .VDD2(VBUF2), .VSS(VSSA), .OUT(OUTA), .OUTN(OUTAN)); \n'.format(i = N))
    f.writelines('BUF_{i}N I5 (.X(XB), .XN(XBN), .VDD2(VBUF), .VSS(VSSA), .OUT(OUTB), .OUTN(OUTBN)); \n'.format(i = N))
    
    
    f.writelines('SAF_XOR_DFF_{i}N I6 (.IN1_P(OUTA), .IN1_N(OUTAN), .IN2_P(OUTB), .IN2_N(OUTBN), .CLK(CLK), .VDD(VDD), .VSS(VSSA), .OUT(OUT), .OUT_N(OUT_N)); \n'.format(i = N))
    f.writelines('DAC_BUF_{i}N I7 (.OUT(DAC_INP), .OUT_N(DAC_INN), .VREFP(VREFP), .VREFN(VSSA), .INP(OUT), .INN(OUT_N)); \n'.format(i = N))
    f.writelines('DAC_{i}N_{j}k I8 (.OUT(VCTRLP), .IN(DAC_INP)); \n'.format(i = N, j = RES))
    f.writelines('DAC_{i}N_{j}k I9 (.OUT(VCTRLN), .IN(DAC_INN)); \n'.format(i = N, j = RES))
    f.writelines('endmodule \n')

In [67]:
print(time.strftime("%Y-%m-%d", time.localtime()))

2022-02-14


In [68]:
#judge if the file exists, if so, clear the old data in it
if os.path.exists("./{fn}/CCO_DSM_{i}N_{j}S_{k}kRin_v{t}.v".format(fn = New_file, i = N, j = S, k = RIN, t = version)):
    #clear old data in top.v
    f = open("./{fn}/CCO_DSM_{i}N_{j}S_{k}kRin_v{t}.v".format(fn = New_file, i = N, j = S, k = RIN, t = version), 'r+')
    f.truncate(0) # need '0' when using r+
    


file1 = "./{fn}/CCO_DSM_{i}N_{j}S_{k}kRin_v{t}.v".format(fn = New_file, i = N, j = S, k = RIN, t = version)
file2 = str(CCO_DSM_top)
file3 = str(CCO_ring)
file3p1 = str(CCO_unit)
file3p2 = str(CCO)
file4 = str(BUF)
file5 = str(SAFF)
file6 = str(SAF_XOR_DFF)
file7 = str(DAC_BUF)
file8 = str(DAC)
file9 = '../DoNotChange/BUF_SAFF_module.v'
file10 = str(RIN_file)
file11 = str(DAC_u)

def merge2(file1, file2):
    f1 = open(file1, 'a+', encoding='utf-8')
    with open(file2, 'r', encoding='utf-8') as f2:
        f1.write('\n')
        for i in f2:
            f1.write(i)


merge2(file1, file2)
merge2(file1, file3)
merge2(file1, file3p1)
merge2(file1, file3p2)
merge2(file1, file4)
merge2(file1, file5)
merge2(file1, file6)
merge2(file1, file7)
merge2(file1, file8)
merge2(file1, file9)
merge2(file1, file10)
merge2(file1, file11)